In [1]:
library(dplyr)
library(tidyr)
library(stringr)
library(kernlab)
library(igraph)
library(Rcpp)
library(inline)
library(mixKernel)
library(Matrix)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘igraph’

The following object is masked from ‘package:tidyr’:

    crossing

The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union


Attaching package: ‘inline’

The following object is masked from ‘package:Rcpp’:

    registerPlugin

Loading required package: mixOmics
Loading required package: MASS

Attaching package: ‘MASS’

The following object is masked from ‘package:dplyr’:

    select

Loading required package: lattice
Loading required package: ggplot2

Attaching package: ‘ggplot2’

The following object is masked from ‘package:kernlab’:

    alpha


Loaded mixOmics 6.3.2

Thank you for us

In [3]:
cppFunction('double xfun2(const NumericMatrix x, const NumericMatrix y, double sigma) {
         int n = x.nrow(), m = y.nrow();
         double temp = 0;
         for (int i = 0; i < n; i++) {
           for (int j = 0; j < m ; j++) {
             temp += exp(-pow(pow(x(i,0)-y(j,0),2)+pow(x(i,1)-y(j,1),2),0.5)/8/sigma)
                      -exp(-pow(pow(x(i,0)-y(j,1),2)+pow(x(i,1)-y(j,0),2),0.5)/8/sigma);
           }
         }
         return temp/8/sigma/3.14159265;
      }')
start.time0 <- Sys.time()
setwd("/gpfs/projects/UITS/IUNI/IMAGENE/Phom_output/thickness_PH_normed_max/PH1_thickTOthin/")
#setwd("/gpfs/projects/UITS/IUNI/IMAGENE/Phom_output/gene_PH_normed_max/PH0_test/")
temp = list.files(pattern="*.csv")
PH0_thickTOthin = lapply(temp, read.csv)
Kmatrix <- diag(length(PH0_thickTOthin))
start.time <- Sys.time()
call.time = 0
for (i in 1:(length(PH0_thickTOthin)-1)){
  for (j in (i):length(PH0_thickTOthin)){
    x <- data.matrix(PH0_thickTOthin[[i]][,1:2])
    y <- data.matrix(PH0_thickTOthin[[j]][,1:2])
    x <- x[is.finite(rowSums(x)),]
    y <- y[is.finite(rowSums(y)),]
    call.time <- call.time + system.time(Kmatrix[i,j] <- xfun2(x, y, 1))[3]
  }
}
end.time <- Sys.time()
Kmatrix[lower.tri(Kmatrix)]  <- t(Kmatrix)[lower.tri(Kmatrix)]
KmatrixPD = nearPD(Kmatrix)
end.time - start.time
call.time/3600
write.table(data.matrix(KmatrixPD$mat), file='/gpfs/projects/UITS/IUNI/IMAGENE/workingdir/yan30/thickPH1thickKernelPD.tsv', quote=FALSE, sep='\t', col.names = F, row.names = F)

Time difference of 5.650636 hours

elapsed 
4.69456